In [ ]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
np.random.seed(42)
from datetime import datetime, timedelta
from time import time

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.models import cn_models
from cryptotrader.utils import array_normalize
from cryptotrader.agents.apriori import TCOTrader

import chainer as cn

import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
# %load_ext line_profiler

In [ ]:
# Simulation Params
test_name = 'EIIETCO_agent'
obs_steps = 40 # Observation steps, number of candles required by the agent for calculations
period = 120 # Observation period in minutes, also trading frequency
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_ZEC", "USDT_DASH"] # Universe, some survivor bias here...
fiat_symbol = 'USDT' # Quote symbol
init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs) # Initial equally distributed portfolio
data_dir = './data' # Data directory for offline testing

In [ ]:
## Environment setup
# Data feed setup
papi = Poloniex()
tapi = BacktestDataFeed(papi, period, pairs=pairs, balance=init_funds, load_dir=data_dir)

# Download new data from the exchange
# tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
#                        start=datetime.timestamp(datetime.utcnow() - timedelta(days=300)))

# # And save it to disk, if you want to
# tapi.save_data(data_dir + '/train')

# Or load data from disk
tapi.load_data('/train')

# Environment setup
env = BacktestEnvironment(period, obs_steps, tapi, fiat_symbol, test_name)
obs = env.reset();

# Setup eval ent
# Or load data from disk
tapi = BacktestDataFeed(papi, period, pairs=pairs, balance=init_funds, load_dir=data_dir)
tapi.load_data('/eval')

# Environment setup
eval_env = BacktestEnvironment(period, obs_steps, tapi, fiat_symbol, test_name)
eval_env.reset();

In [ ]:
# NN params
timesteps = obs.shape[0] - 1
n_filters_in = 8
n_filters_out = 64
lr = 1e-3
lr_decay_period = 1000
l2_reg = 1e-6
target_type = 'classifier'
train_epochs = 1000000
test_interval = 500
test_epochs = 128
batch_size = 4
save_dir = './save/'

In [ ]:
# NN setup
eiie = cn_models.EIIE(timesteps, n_filters_in, n_filters_out)
optimizer = cn.optimizers.Adam(alpha=lr)
l2_reg = cn.optimizer.WeightDecay(l2_reg)
optimizer.setup(eiie)
optimizer.add_hook(l2_reg)
name = 'eiie'

In [ ]:
# Train NN
cn.serializers.load_npz(save_dir + 'eiie.npz', obj=eiie)

cn_models.train_nn(eiie, env, eval_env, optimizer, batch_size, lr_decay_period,
                   train_epochs, test_interval, test_epochs, target_type, save_dir, name)

In [ ]:
# Agent Setup
cn.serializers.load_npz(save_dir + 'eiie.npz', obj=eiie)
agent = TCOTrader(predictor=eiie.predict, toff=0.001)

# Optimization params
nb_steps = 100
batch_size = 1
nb_max_episode_steps = 7

# Params search space
search_space = {'toff': [1e-8, 1e-1]}

# Optimization session, this may take some time
opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

# Run on training data
agent.test(eval_env, verbose=True)
# Display results
eval_env.plot_results();